**Table of contents**<a id='toc0_'></a>    
- 1. [Problem 1: Production economy and CO2 taxation](#toc1_)    
- 2. [Problem 2: Career choice model](#toc2_)    
- 3. [Problem 3: Barycentric interpolation](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [26]:
import numpy as np
from types import SimpleNamespace
from scipy.optimize import minimize_scalar, root
import matplotlib.pyplot as plt

## 1. <a id='toc1_'></a>[Problem 1: Production economy and CO2 taxation](#toc0_)

Consider a production economy with two firms indexed by $j \in \{1,2\}$. Each produce its own good. They solve

$$
\begin{align*}
\max_{y_{j}}\pi_{j}&=p_{j}y_{j}-w_{j}\ell_{j}\\\text{s.t.}\;&y_{j}=A\ell_{j}^{\gamma}.
\end{align*}
$$

Optimal firm behavior is

$$
\begin{align*}
\ell_{j}^{\star}(w,p_{j})&=\left(\frac{p_{j}A\gamma}{w}\right)^{\frac{1}{1-\gamma}} \\
y_{j}^{\star}(w,p_{j})&=A\left(\ell_{j}^{\star}(w,p_{j})\right)^{\gamma}
\end{align*}
$$

The implied profits are

$$
\pi_{j}^*(w,p_{j})=\frac{1-\gamma}{\gamma}w\cdot\left(\frac{p_{j}A\gamma}{w}\right)^{\frac{1}{1-\gamma}}
$$

A single consumer supplies labor, and consumes the goods the firms produce. She also recieves the implied profits of the firm.<br>
She solves:

$$
\begin{align*}
U(p_1,p_2,w,\tau,T) = \max_{c_{1},c_{2},\ell} & \log(c_{1}^{\alpha}c_{2}^{1-\alpha})-\nu\frac{\ell^{1+\epsilon}}{1+\epsilon} \\
\text{s.t.}\,\,\,&p_{1}c_{1}+(p_{2}+\tau)c_{2}=w\ell+T+\pi_{1}^*(w,p_{1})+\pi_{2}^*(w,p_{2})
\end{align*}
$$

where $\tau$ is a tax and $T$ is lump-sum transfer. <br>
For a given $\ell$, it can be shown that optimal behavior is

$$
\begin{align*}
c_{1}(\ell)&=\alpha\frac{w\ell+T+\pi_{1}^*(w,p_{1})+\pi_{2}^*(w,p_{2})}{p_{1}} \\
c_{2}(\ell)&=(1-\alpha)\frac{w\ell+T+\pi_{1}^*(w,p_{1})+\pi_{2}^*(w,p_{2})}{p_{2}+\tau} \\
\end{align*}
$$
Such that optimal behavior is:
$$
\ell^* = \underset{\ell}{\arg\max} \log(\left(c_{1}(\ell)\right)^{\alpha}\cdot \left(c_{2}(\ell)\right)^{1-\alpha})-\nu\frac{\ell^{1+\epsilon}}{1+\epsilon} 
$$
With optimal consumption:
$$
\begin{align*}
c_1^*=c_{1}(\ell^*) \\
c_2^*=c_{2}(\ell^*)\\
\end{align*}
$$


The government chooses $\tau$ and balances its budget so $T=\tau c_2^*$. We initially set $\tau,T=0$.

Market clearing requires:

1. Labor market: $\ell^* = \ell_1^* + \ell_2^*$
1. Good market 1: $c_1^* = y_1^*$
1. Good market 2: $c_2^* = y_2^*$


**Question 1:** Check market clearing conditions for $p_1$ in `linspace(0.1,2.0,10)` and $p_2$ in `linspace(0.1,2.0,10)`. We choose $w=1$ as numeraire.

In [27]:
#Setting parameters
par = SimpleNamespace()

#Firms
par.A = 1.0
par.gamma = 0.5

#Households
par.alpha = 0.3
par.nu = 1.0
par.epsilon = 2.0

#Government
par.tau = 0.0
par.T = 0.0

#Question 3
par.kappa = 0.1

In [28]:
#Firm functions
def labor_demand(p):
    return (p * par.A * par.gamma) ** (1 / (1 - par.gamma))

def good_supply(l):
    return par.A * l ** par.gamma

def profit(p):
    return ((1 - par.gamma) / par.gamma) * (p * par.A * par.gamma) ** (1 / (1 - par.gamma))

#Consumer functions
def good_demand_1(p1, p2, l):
    income = l + par.T + profit(p1) + profit(p2)
    return par.alpha * income / p1

def good_demand_2(p1, p2, l):
    income = l + par.T + profit(p1) + profit(p2)
    return (1 - par.alpha) * income / (p2 + par.tau)

def labor_supply(p1, p2):
    #Defining the objective function
    def objective(l):
        c1 = good_demand_1(p1, p2, l)
        c2 = good_demand_2(p1, p2, l)
        return np.log(c1 ** par.alpha * c2 ** (1 - par.alpha)) - par.nu * l ** (1 + par.epsilon) / (1 + par.epsilon)
    
    #Find the labor that maximizes the objective
    result = minimize_scalar(lambda l: -objective(l), bounds=(0, 10), method='bounded')
    return result.x

#Check market clearing conditions
for p1 in np.linspace(0.1, 2.0, 10):
    for p2 in np.linspace(0.1, 2.0, 10):
        #Firm labor demand
        labor_demand_1 = labor_demand(p1)
        labor_demand_2 = labor_demand(p2)

        #Firm good supply
        good_supply_1 = good_supply(labor_demand_1)
        good_supply_2 = good_supply(labor_demand_2)

        #Consumer labor supply
        l = labor_supply(p1, p2)

        #Consumer good demand
        c1 = good_demand_1(p1, p2, l)
        c2 = good_demand_2(p1, p2, l)

        #Market clearing conditions
        LM = l - labor_demand_1 - labor_demand_2
        GM1 = good_supply_1 - c1
        GM2 = good_supply_2 - c2

        #Printing results
        print(f'(p1,p2) = ({p1:.2f},{p2:.2f}) -> Excess supply on Labor Market: {LM:.2f}, Good Market 1: {GM1:.2f}, Good Market 2: {GM2:.2f}')

(p1,p2) = (0.10,0.10) -> Excess supply on Labor Market: 0.99, Good Market 1: -2.96, Good Market 2: -6.97
(p1,p2) = (0.10,0.31) -> Excess supply on Labor Market: 0.96, Good Market 1: -3.00, Good Market 2: -2.13
(p1,p2) = (0.10,0.52) -> Excess supply on Labor Market: 0.91, Good Market 1: -3.09, Good Market 2: -1.14
(p1,p2) = (0.10,0.73) -> Excess supply on Labor Market: 0.82, Good Market 1: -3.23, Good Market 2: -0.68
(p1,p2) = (0.10,0.94) -> Excess supply on Labor Market: 0.70, Good Market 1: -3.42, Good Market 2: -0.38
(p1,p2) = (0.10,1.16) -> Excess supply on Labor Market: 0.56, Good Market 1: -3.66, Good Market 2: -0.17
(p1,p2) = (0.10,1.37) -> Excess supply on Labor Market: 0.40, Good Market 1: -3.95, Good Market 2: -0.00
(p1,p2) = (0.10,1.58) -> Excess supply on Labor Market: 0.20, Good Market 1: -4.31, Good Market 2: 0.14
(p1,p2) = (0.10,1.79) -> Excess supply on Labor Market: -0.01, Good Market 1: -4.73, Good Market 2: 0.27
(p1,p2) = (0.10,2.00) -> Excess supply on Labor Market: 

**Question 2:** Find the equilibrium prices $p_1$ and $p_2$.<br>
*Hint: you can use Walras' law to only check 2 of the market clearings*

In [29]:
#Define function that expresses excess demand given prices
def market_clearing(prices):
    p1, p2 = prices
    labor_demand_1 = labor_demand(p1)
    labor_demand_2 = labor_demand(p2)
    good_supply_1 = good_supply(labor_demand_1)
    l = labor_supply(p1, p2)
    c1 = good_demand_1(p1, p2, l)
    LM = l - labor_demand_1 - labor_demand_2
    GM1 = good_supply_1 - c1
    return [LM, GM1]

#Finding equilibrium prices, with (p1,p2) = (1.0,1.0) as initial guess
result = root(market_clearing, [1.0, 1.0])

#Printing solution
if result.success:
    equilibrium_prices = result.x
    print(f'Equilibrium prices: p1 = {equilibrium_prices[0]:.2f}, p2 = {equilibrium_prices[1]:.2f}')
else:
    print('Equilibrium not found')

Equilibrium prices: p1 = 0.98, p2 = 1.49


Assume the government care about the social welfare function:

$$
SWF = U - \kappa y_2^*
$$

Here $\kappa$ measures the social cost of carbon emitted by the production of $y_2$ in equilibrium.

**Question 3:** What values of $\tau$ and (implied) $T$ should the government choose to maximize $SWF$?

In [30]:
#Function to find equilibrium values for given tau
def find_equilibrium(tau, T):
    par.tau = tau
    par.T = T

    initial_prices = [1.0, 1.0]
    result = root(market_clearing, initial_prices)

    if result.success:
        optimal_prices = result.x
        l = labor_supply(optimal_prices[0], optimal_prices[1])
        c2 = good_demand_2(optimal_prices[0], optimal_prices[1], l)
        return c2, optimal_prices
    else:
        raise ValueError('Equilibrium prices not found')

#Social welfare function given tau
def social_welfare(tau):
    # Initial guess for T
    T = 0
    tolerance = 1e-6
    max_iterations = 10
    iteration = 0
    
    #Updating T untill converged
    while iteration < max_iterations:
        iteration += 1
        previous_T = T
        c2, optimal_prices = find_equilibrium(tau, T)
        T = tau * c2
        
        if abs(T - previous_T) < tolerance:
            break

    #Final equilibrium with converged T
    c2, optimal_prices = find_equilibrium(tau, T)
    l = labor_supply(optimal_prices[0], optimal_prices[1])
    c1 = good_demand_1(optimal_prices[0], optimal_prices[1], l)
    
    #Utility function
    U = np.log(c1 ** par.alpha * c2 ** (1 - par.alpha)) - par.nu * l ** (1 + par.epsilon) / (1 + par.epsilon)
    #Social welfare function
    SWF = U - par.kappa * c2
    return -SWF  #We minimize the negative of SWF to maximize SWF

#Finding optimal tau
result_tau = minimize_scalar(social_welfare, bounds=(0, 1), method='bounded')

#Printing solution
if result_tau.success:
    optimal_tau = result_tau.x
    print(f'Optimal tau is {optimal_tau:.2f}, which leads to SWF = {-result_tau.fun:.2f}')
else:
    print('Optimal tau not found')

Optimal tau is 0.17, which leads to SWF = -0.66
